In [858]:
import re
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd

### Import datasets

In [859]:
filenames = ['./data/candidates_df.csv', './data/tvo_df.csv']
dataframes = [pd.read_csv(df, sep='\t') for df in filenames]
candidates_df, tvo_df = dataframes[0], dataframes[1]

### Transform Candidates dataframe

In [860]:
# adding space between Surname and Name
candidates_df["Прізвище, ім'я, по батькові"] = candidates_df["Прізвище, ім'я, по батькові"].str.replace(r'([а-яєґії])([А-ЯҐЄІЇ]+)', r'\1 \2', regex=True)

In [861]:
# party names in uppercase
candidates_df['Партія'] = candidates_df['Партія'].str.upper()

In [862]:
# split birth and origin
candidates_df = candidates_df.join(candidates_df['Дата та місце народження'].str.extract('(?P<Дата_народження>[\d{2}\.\d{2}\.\d{4}]{10})(?P<Місце_народження>[\s\S]+)', expand=True))
candidates_df = candidates_df.drop(columns = ['Дата та місце народження'])


In [863]:
# delete dots after birthday
candidates_df['Дата_народження'] = candidates_df['Дата_народження'].str.rstrip('\.$')

In [864]:
#changing column names
candidates_df = candidates_df.rename(columns={'№ у терито-ріальному списку': '№ у територіальному списку', 'Дата_народження': 'Дата народження', 'Місце_народження': 'Місце народження'})

In [865]:
# adding space in birthplace
candidates_df['Місце народження'] = candidates_df['Місце народження'].str.replace(r'(\.)([А-ЯҐЄІЇ]+)', r'\1 \2', regex=True)

### Transform Elected Candidates dataframe

In [866]:
# changing name column in elected
tvo_df = tvo_df.rename(columns={"Прізвище, ім’я, по батькові": "Прізвище, ім'я, по батькові"})

In [867]:
# adding birthday column in elected
tvo_df['Дата народження'] = tvo_df['Відомості про обраного депутата'].str.extract('(\d{2}.\d{2}.\d{4})')

### Merge Candidates and Elected dfs

In [868]:
# merge two columns based on name, TVO, birthday
new_df = candidates_df.merge(tvo_df,how='left', left_on=["Прізвище, ім'я, по батькові", "№ ТВО, за яким закріплено", "Дата народження"], right_on=["Прізвище, ім'я, по батькові", "№ ТВО, за яким закріплено", "Дата народження"])

In [869]:
# drop columns with duplicate information
new_df = new_df.drop(columns = ["Відомості про обраного депутата", "Голосів ЗА", "% від квоти"])

### Add 'Status' column

In [870]:
# New column based on the election status
tvo_df['Виборчий округ, в якому обрано'].isnull().values.any()
new_df['Статус'] = new_df["Виборчий округ, в якому обрано"].isna().apply(lambda x: "обрано" if x==0 else 'не обрано')

### Clean joined table

In [871]:
# remove dots in the beginning of birthplace
new_df['Місце народження'] = new_df['Місце народження'].str.lstrip('^.')

In [872]:
# standardize region pattern
region_pattern = '|'.join([' айо', ' бл.', ' області', ' область', 'обл'])
new_df['Місце народження'] = new_df['Місце народження'].str.replace(region_pattern, ' обл.')

In [873]:
# remove multiple dots
new_df['Місце народження'] = new_df['Місце народження'].str.replace('[.]{2,}', '.')

In [874]:
# reindex dataframe
column_names = ["Прізвище, ім'я, по батькові", 'Дата народження', 'Місце народження', 'Відомості', 'Партія','Область', 'Міська рада', '№ у єдиному списку', '№ ТВО, за яким закріплено', '№ у територіальному списку', 'Кількість отриманих голосів', '% голосів від квоти', 'Черговість кандидата за результатами','Статус', 'Виборчий округ, в якому обрано']
new_df = new_df.reindex(columns = column_names)

### Save dataframe

In [875]:
# save output
new_df.to_csv("./data/Joined_table.csv", sep='\t', index=False)